<a href="https://colab.research.google.com/github/MNeilDiane/Augustine/blob/master/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pandas as pd
import numpy as np
from google.colab import drive 
!gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip
!unzip data.zip
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
data = pd.read_csv('./train.csv', encoding = 'big5')
data.head(10)
#读入数据

Downloading...
From: https://drive.google.com/uc?id=1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm
To: /content/data.zip
100% 177k/177k [00:00<00:00, 1.67MB/s]
Archive:  data.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


,日期,測站,測項,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,12,15,17,20,22,22,22,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,0.59,0.52,0.41,0.4,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,0.2,0.18,0.12,0.11,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,4.2,2.9,3.4,3,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014/1/1,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,7.8,15,21,14,11,14,12,11,11,22,28,19,12,8.1,7,6.9,6
6,2014/1/1,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,9.9,22,29,18,14,17,15,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014/1/1,豐原,O3,16,30,27,23,24,28,24,22,21,29,44,58,50,57,65,64,51,34,33,34,37,38,38,36
8,2014/1/1,豐原,PM10,56,50,48,35,25,12,4,2,11,38,56,64,56,57,52,51,66,85,85,63,46,36,42,42
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,20,19,30,41,44,33,37,36,45,42,49,45,44,41,30,24,13


In [ ]:
#丢弃前两列
data = data.iloc[:,3:]
#将降水设为0
data[data == 'NR'] = 0
#将dataframe转为numpy的数组
raw_data = data.to_numpy()


In [ ]:
month_data={}
for month in range(12):
  sample = np.empty([18,480])
  for day in range(20):
    sample[:,day * 24:(day + 1) * 24]=raw_data[18 * (20 * month +day) : 18 * (20 * month + day +1),:]
  month_data[month] = sample

In [ ]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)


[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [ ]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x


array([[-1.35825331, -1.35883937, -1.359222  , ...,  0.26650729,
         0.2656797 , -1.14082131],
       [-1.35825331, -1.35883937, -1.51819928, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [-1.35825331, -1.51789368, -1.67717656, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [-0.88092053, -0.72262212, -0.56433559, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [-0.7218096 , -0.56356781, -0.72331287, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [-0.56269867, -0.72262212, -0.88229015, ..., -0.38950555,
        -0.10906991,  0.07797893]])

In [ ]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))


[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

In [ ]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 2
iter_time = 10000
adagrad = np.zeros([dim, 1])
eps = 0.0000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w


0:27.071214829194115
100:7.036024096840856
200:6.483255417799993
300:6.252630186180072
400:6.118935670779613
500:6.031127888033844
600:5.968813383252804
700:5.9222125435530195
800:5.886025131119694
900:5.85711859324193
1000:5.833520263644294
1100:5.8139255749844825
1200:5.7974365745994625
1300:5.783413656387992
1400:5.771387304604553
1500:5.761003528132467
1600:5.751989053249454
1700:5.744128505570382
1800:5.73724907190655
1900:5.731209942287559
2000:5.725894874614569
2100:5.721206842196417
2200:5.717064099156934
2300:5.713397230483445
2400:5.710146899367619
2500:5.7072620978233015
2600:5.704698767242115
2700:5.702418695626139
2800:5.700388625136406
2900:5.698579521941407
3000:5.696965973062231
3100:5.69552568385954
3200:5.694239056203541
3300:5.693088832007825
3400:5.692059790221199
3500:5.691138487916632
3600:5.690313038038054
3700:5.68957291783411
3800:5.688908803142446
3900:5.688312424574037
4000:5.687776442345438
4100:5.687294337062803
4200:5.68686031420762
4300:5.686469220435182


array([[ 2.13740269e+01],
       [ 3.74483036e-01],
       [-8.47203292e-01],
       [ 1.11951314e+00],
       [-1.66196572e+00],
       [-3.77155348e-02],
       [ 1.01169550e+00],
       [-3.75868393e-01],
       [-1.68666156e+00],
       [ 2.01000390e+00],
       [-3.31146797e-01],
       [ 1.41416603e-01],
       [ 7.74727234e-02],
       [ 9.84332458e-02],
       [-3.03939552e-02],
       [-2.41105306e-02],
       [-1.79578021e-01],
       [ 1.64950759e-01],
       [ 5.18134231e-01],
       [ 5.24168977e-02],
       [-2.56450586e-02],
       [ 6.62135505e-02],
       [-1.59350344e-01],
       [ 1.52594043e-01],
       [-2.02825444e-02],
       [-1.63422996e-01],
       [ 7.76999647e-02],
       [ 4.03539981e-01],
       [-3.19172383e-01],
       [ 3.75299201e-01],
       [-2.61455612e-01],
       [ 2.97288083e-01],
       [ 3.09195197e-01],
       [-5.20256867e-01],
       [ 9.36602030e-02],
       [ 1.82534145e-01],
       [ 8.42624353e-02],
       [ 7.78153318e-02],
       [ 5.4

In [ ]:
# 读入测试数据test.csv
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
# 丢弃前两列，需要的是从第3列开始的数据
test_data = testdata.iloc[:, 2:]
# 把降雨为NR字符变成数字0
test_data[test_data == 'NR'] = 0
# 将dataframe变成numpy数组
test_data = test_data.to_numpy()
# 将test数据也变成 240 个维度为 18 * 9 + 1 的数据。
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3718: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

In [ ]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y


array([[  6.57496077],
       [ 18.20658777],
       [ 23.94773146],
       [  7.70772963],
       [ 27.01518663],
       [ 22.25787871],
       [ 23.87907681],
       [ 30.4711901 ],
       [ 16.77180283],
       [ 59.66370762],
       [ 11.72189443],
       [  9.29665427],
       [ 63.23837984],
       [ 53.24409927],
       [ 22.19387016],
       [ 12.20059631],
       [ 32.40284606],
       [ 66.76878265],
       [ -0.53147513],
       [ 17.12993079],
       [ 41.781914  ],
       [ 72.10101114],
       [  9.30404897],
       [ 17.84077377],
       [ 14.75244394],
       [ 37.77965388],
       [ 14.5390974 ],
       [ 69.73550913],
       [  7.2219375 ],
       [ 55.40984193],
       [ 24.50576262],
       [  8.49405234],
       [  2.50955413],
       [ 18.6756775 ],
       [ 27.76474103],
       [ 37.20908216],
       [ 43.23766046],
       [ 29.8274036 ],
       [ 42.21773613],
       [ 35.13684828],
       [  7.57613055],
       [ 41.21160532],
       [ 30.57744616],
       [ 50

In [ ]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)


['id', 'value']
['id_0', 6.574960767520195]
['id_1', 18.20658776714244]
['id_2', 23.94773145719033]
['id_3', 7.707729625341625]
['id_4', 27.0151866257063]
['id_5', 22.25787870668416]
['id_6', 23.879076811695924]
['id_7', 30.471190101219502]
['id_8', 16.771802829249634]
['id_9', 59.66370762477012]
['id_10', 11.721894428717132]
['id_11', 9.29665427178614]
['id_12', 63.23837984350006]
['id_13', 53.24409927003313]
['id_14', 22.19387015994616]
['id_15', 12.200596309868422]
['id_16', 32.402846055731864]
['id_17', 66.7687826529527]
['id_18', -0.5314751251021796]
['id_19', 17.129930787298395]
['id_20', 41.78191400279282]
['id_21', 72.10101114257186]
['id_22', 9.304048970628157]
['id_23', 17.840773771578313]
['id_24', 14.752443935459397]
['id_25', 37.77965388387494]
['id_26', 14.539097398438336]
['id_27', 69.73550913002622]
['id_28', 7.221937501078756]
['id_29', 55.40984193285242]
['id_30', 24.505762618010493]
['id_31', 8.494052339918008]
['id_32', 2.509554134022701]
['id_33', 18.67567750287756